In [3]:
import pandas as pd
import numpy as np
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
import folium
import json
import pydeck as pdk
import geopandas as gpd

In [2]:
yp_df = pd.read_excel("../../data/moving_population/ingu_hour_yeungdeungpo.xlsx",sheet_name=0)

In [3]:
mp_df = pd.read_excel("../../data/moving_population/ingu_hour_mapo.xlsx",sheet_name=0)

In [4]:
ingu_var = [x for x in yp_df.columns if '_ingu' in x and len(x)>13]
print(len(ingu_var))
yp_df_2 = pd.melt(yp_df,['hour','latitude','longitude','gu_name','dong_name'],value_vars=ingu_var,var_name='성별연령대',value_name='인구')
yp_df_2.head()
yp_df_2['연령대'] = yp_df_2['성별연령대'].str.slice(stop=7)
yp_df_2['성별'] = yp_df_2['성별연령대'].str.slice(start=8)
yp_df_2 = yp_df_2.drop("성별연령대", axis=1)
yp_df_2['성별'] = yp_df_2.apply(lambda row:row['성별'].split('_')[0],axis=1)
yp_df_2.head()

18


,hour,latitude,longitude,gu_name,dong_name,인구,연령대,성별
0,0,37.510243,126.904651,영등포구,영등포본동,0.185763,under10,male
1,0,37.516164,126.917051,영등포구,영등포본동,0.112386,under10,male
2,0,37.512080,126.911425,영등포구,영등포본동,0.000000,under10,male
3,0,37.511581,126.901812,영등포구,영등포본동,0.417325,under10,male
4,0,37.514795,126.913667,영등포구,영등포본동,0.076697,under10,male


In [8]:
yp_df_2.groupby('dong_name')['latitude'].describe()

,count,mean,std,min,25%,50%,75%,max
dong_name,,,,,,,,
당산1동,128736.0,37.523915,0.002287,37.519236,37.521928,37.523744,37.525955,37.529119
당산2동,265320.0,37.533928,0.004710,37.526030,37.530034,37.533226,37.537741,37.544031
대림1동,86832.0,37.493676,0.002038,37.489070,37.492214,37.493588,37.495363,37.497191
대림2동,99360.0,37.489601,0.002182,37.485439,37.487713,37.489491,37.491287,37.494447
대림3동,160704.0,37.500029,0.003692,37.492621,37.497139,37.499851,37.502557,37.508383
도림동,101088.0,37.509134,0.001623,37.504353,37.507972,37.509314,37.510257,37.512071
문래동,310140.0,37.516519,0.002813,37.508830,37.514282,37.516532,37.518739,37.522736
신길1동,123552.0,37.512423,0.003080,37.505796,37.509868,37.512128,37.515268,37.518422
신길3동,91152.0,37.506210,0.001609,37.503009,37.504846,37.506173,37.507539,37.510271


In [ ]:
mp_df_2 = pd.melt(mp_df,['hour','latitude','longitude','gu_name','dong_name'],value_vars=ingu_var,var_name='성별연령대',value_name='인구')
mp_df_2.head()
mp_df_2['연령대'] = mp_df_2['성별연령대'].str.slice(stop=7)
mp_df_2['성별'] = mp_df_2['성별연령대'].str.slice(start=8)
mp_df_2 = mp_df_2.drop("성별연령대", axis=1)
mp_df_2.head()
mp_df_2['성별'] = mp_df_2.apply(lambda row:row['성별'].split('_')[0],axis=1)
mp_df_2.head()

In [14]:

yp_df_3 

,hour,latitude,longitude,gu_name,dong_name,인구,연령대,성별
0,0,37.510243,126.904651,영등포구,영등포본동,0.185763,under10,male_ingu
1,0,37.516164,126.917051,영등포구,영등포본동,0.112386,under10,male_ingu
2,0,37.512080,126.911425,영등포구,영등포본동,0.000000,under10,male_ingu
3,0,37.511581,126.901812,영등포구,영등포본동,0.417325,under10,male_ingu
4,0,37.514795,126.913667,영등포구,영등포본동,0.076697,under10,male_ingu
...,...,...,...,...,...,...,...,...
3805663,23,37.495322,126.895154,영등포구,대림3동,0.203570,upper80,female_ingu
3805664,23,37.493985,126.897993,영등포구,대림3동,0.923317,upper80,female_ingu
3805665,23,37.500291,126.897376,영등포구,대림3동,0.568758,upper80,female_ingu
3805666,23,37.503882,126.894519,영등포구,대림3동,0.110731,upper80,female_ingu


In [21]:
yp_df_3 = yp_df_2.copy()
yp_df_3 = yp_df_3.apply(lambda row:row['성별'].split('_')[0],axis=1)

In [24]:
yp_df_2['성별'] = yp_df_3

In [28]:
yp_df_2.to_csv('post_processed_yp.csv')

In [30]:
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    yp_df_2['인구'].min(),
    yp_df_2['인구'].max())
colormap

In [82]:
groupby_dong = []
for _hour in range(24):
    temp_df = yp_df_2[yp_df_2['hour']==_hour].groupby('dong_name').sum()['인구'].to_frame()
    temp_df['dong_name'] = temp_df.index
    temp_df['hour'] = _hour
    temp_df['index'] = range(temp_df.shape[0])
    temp_df = temp_df.set_index('index')
    groupby_dong.append(temp_df)
    
move_groupby_dong = []
for x in groupby_dong:
    temp_df = x.copy()
    temp_df['인구'] = x['인구']-groupby_dong[3]['인구']
    move_groupby_dong.append(temp_df)


In [83]:
temp_df

,인구,dong_name,hour
index,,,
0,216.375021,당산1동,23
1,8.295235,당산2동,23
2,-483.624985,대림1동,23
3,-116.165969,대림2동,23
4,-27.835558,대림3동,23
5,-441.783650,도림동,23
6,-505.503337,문래동,23
7,-435.505765,신길1동,23
8,-275.096423,신길3동,23


In [4]:
geo_path = 'geodata/seoul_yp_mp_map.geojson'
geo_str = json.load(open(geo_path, encoding='utf-8'))
# geo_str

In [10]:
df = gpd.read_file(geo_path)
df.to_csv("geojson_excel.csv")


In [12]:
df2 = pd.read_csv("./geojson_excel.csv")
df2

,Unnamed: 0,OBJECTID,adm_nm,adm_cd,adm_cd2,gu,dong,geometry
0,0,206,서울특별시 마포구 용강동,1114059,1144059000,마포구,용강동,MULTIPOLYGON (((126.9445281888074 37.545147999...
1,1,207,서울특별시 마포구 대흥동,1114060,1144060000,마포구,대흥동,MULTIPOLYGON (((126.9468122045961 37.556849451...
2,2,208,서울특별시 마포구 염리동,1114061,1144061000,마포구,염리동,MULTIPOLYGON (((126.949681604116 37.5438189266...
3,3,209,서울특별시 마포구 신수동,1114063,1144063000,마포구,신수동,MULTIPOLYGON (((126.9407314087531 37.546613890...
4,4,210,서울특별시 마포구 서교동,1114066,1144066000,마포구,서교동,MULTIPOLYGON (((126.9135347792953 37.549483237...
5,5,211,서울특별시 마포구 합정동,1114068,1144068000,마포구,합정동,MULTIPOLYGON (((126.9135347792953 37.549483237...
6,6,212,서울특별시 마포구 망원1동,1114069,1144069000,마포구,망원1동,MULTIPOLYGON (((126.9087975895597 37.557418691...
7,7,213,서울특별시 마포구 망원2동,1114070,1144070000,마포구,망원2동,MULTIPOLYGON (((126.9043048704066 37.562397559...
8,8,214,서울특별시 마포구 연남동,1114071,1144071000,마포구,연남동,MULTIPOLYGON (((126.9271989081632 37.561829123...
9,9,215,서울특별시 마포구 성산1동,1114072,1144072000,마포구,성산1동,MULTIPOLYGON (((126.9176020782441 37.558838154...


In [84]:
# 서울시 중심부의 위도, 경도 입니다. 
center = [37.541, 126.986] 
x,y = yp_df['latitude'].mean(),yp_df['longitude'].mean()
# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다. 
m = folium.Map(location=[x,y], zoom_start=13)

# # Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(geo_data=geo_str,
                  data=move_groupby_dong[2], columns=('dong_name','인구'), key_on='feature.properties.dong', fill_color=colormap, legend_name='10 인구수', ).add_to(m)

# 맵 m을 출력합니다. 

m

ValueError: color should be a string, not a <class 'branca.colormap.LinearColormap'>.